In [16]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from numpy import *

from ggplot import *
from scipy.stats import logistic
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import *
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


from linearmodels.iv import IV2SLS
from scipy import stats
from scipy.stats import trim_mean #trimmed mean
from scipy.stats import * #chi2_contingency로 카이제곱통계량계산
from scipy.stats import norm

import keyword
import random

#open api 불러올 때 필요한 것
from urllib.request import urlopen #request package 설치
from bs4 import BeautifulSoup #bs4 package 설치

# 그래프에서 한글 깨지지 않기 위헤 사용함
from matplotlib import rcParams
from matplotlib import font_manager,rc

rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='Malgun Gothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)

import FinanceDataReader as fdr
import plotly.graph_objects as go
import plotly.subplots as ms
import math



# LN10 : 회귀분석(1)
## 1 회귀분석 방법: simple linear regression
- 0513 후반 필기하기

## 2 소득과 순자산 간 관계
- 

In [17]:
df=pd.read_csv('c:/temp/household_2020a_1_30.csv')
df.head()

,Unnamed: 0,year,metro,id,sex,age,number,education,marriage,asset,debt,income,income_d,industry,job,house
0,19,2020,G1,10001102,1,35,1,5,1,5252,0,1089,1033,L,5,2
1,134,2020,G2,11800271,2,30,1,6,1,17920,5000,3756,3032,O,4,2
2,157,2020,G1,12000102,1,30,1,6,1,540,0,2004,1812,G,5,3
3,190,2020,G1,12600311,1,36,1,6,1,1500,6280,5802,3959,C,2,3
4,191,2020,G1,12600321,2,31,1,6,1,1610,0,3032,1197,J,2,5


In [18]:
# 전체 데이터에서 30대 1인가구 데이터만 모아 놓은 것
# 몇명?
df.shape

(388, 16)

In [19]:
# income: 경상소득, _d: 가처분소득
df1=df[['income_d','asset']]
df1.head()

,income_d,asset
0,1033,5252
1,3032,17920
2,1812,540
3,3959,1500
4,1197,1610


In [20]:
df1['income']=df1['income_d']/10000 # 억 단위로 변환
df1['net_asset']=df1['asset']/10000 # 억 단위로 변환
df1.head()

,income_d,asset,income,net_asset
0,1033,5252,0.1033,0.5252
1,3032,17920,0.3032,1.7920
2,1812,540,0.1812,0.0540
3,3959,1500,0.3959,0.1500
4,1197,1610,0.1197,0.1610


In [21]:
# 두 변수 간 상관계수를 계산해보자
# 상관계수 : -1~1, 상관관계 정도
df1[['income','net_asset']].corr() # 매우 높은 야의 상관관계가 나타난다

,income,net_asset
income,1.000000,0.367209
net_asset,0.367209,1.000000


In [22]:
# 산포도와 회귀직선... 필기 하기
# 상관계수 0.37의 의미
# x변수인 소득이 1 standard devation 증가하면 y변수인 순자산이 0.37 std 증가한다느 뜻
# 설명
np.std(df1['income']) # X 변수의 표준편차를 계산한다

0.19005961153980735

In [23]:
np.std(df1['net_asset']) # Y 변수의 표준편차를 계산한다

1.6586041054791034

In [24]:
# X 변수가 0.19억 증가하면, Y변수는 0.37*1.65 증가한다
print(0.37*1.65)

0.6104999999999999


In [25]:
# 0.61억, 6100만원

In [26]:
# 회귀직선의 기울기
# 회귀를 돌리지 않아도, simple linear regression에서의 beta_1(기울기) 값은 아마 다음과 같이 계산될 것이다.
# X, Y변수의 sd와 두 변수의 corr만 알면 기울기 구할 수 있다
0.37*(1.6586041054791034/0.19005961153980735)

3.2289002069160513

## 3 회귀직선
- 필기하기

## 4 30대 1인가구의 소득과 자산 간 회귀 분석
- 필기 하기


In [27]:
# Simple Linear Regression 추정 결과: alpha_hat, beta_hat을 얻는다

# Y: df1['net_asset'], X: df1['income']
# ols 함수에서는 formula 방식으로 변수를 표현
# y, x, 변수 순서로 쓰고 ~로 이어준다
# 그 두 변수가 어느 데이터프레임에 있는지는 data= 하고 쓴다

asset_model=ols('net_asset~income',data=df1).fit()
print(asset_model.summary())

                            OLS Regression Results                            
Dep. Variable:              net_asset   R-squared:                       0.135
Model:                            OLS   Adj. R-squared:                  0.133
Method:                 Least Squares   F-statistic:                     60.16
Date:                Sun, 09 Jun 2024   Prob (F-statistic):           7.87e-14
Time:                        16:33:53   Log-Likelihood:                -718.77
No. Observations:                 388   AIC:                             1442.
Df Residuals:                     386   BIC:                             1449.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.5051      0.145      3.494      0.0

In [28]:
# incoe coef: 3.20 = 기울기가 3.20
# 절편은 0.50

> 위 추정 해석
> - 상수항 : 0.50 = 5000만원 = 소득이 없는 사람 (x=0)도 자산 5000만원은 있는 것
> - 기울기 : 3.2 = x가 1억 증가하면 자산은 3.2억 증가한다. 
> - 그 뒤 강의 필기하기

- 해석 이어서

In [29]:
# 소득이 5000만원인 1인 가구의 자산 평균은?
0.5051+3.2045*0.5

2.10735

In [32]:
# 2억 천만원 정도, 2.1억 정도

> -그 중에 순자산이 2억 이상일 확률은?
> - 그걸 알라면 표준편차 알아ㅑㅇ 함 
> - 표준편차 어케 계산하냐??
> - 이 앞뒤로 필기
> - 여기서 표준편차는 RMSE로 정의 함

In [35]:
# 회귀직선의 표준편차 = 1.54
np.sqrt(1-0.367**2)*np.std(df1['net_asset'])

1.5428682725699487

In [36]:
# RMSE = 1.54

In [38]:
# 표준화 
z=(2-2.1)/1.54
z

-0.06493506493506498

In [39]:
# Pr(z>-0.064) 일 확률?
p_value=1-norm.cdf(z)
print(p_value)

0.5258871491417278


In [40]:
# 52% = 소득이 5000만원인 사람은 평균자산은 2.1억이지만, 자산이 2억 이상일 확률

In [41]:
# z, rmse 이런거 이번처럼 찾지 말고, 파이썬으로 하는 법 다음시간에 배울 것